In [ ]:
import itertools
import numpy as np

# def calculate_scores_for_pcd(pcd, SQ_type_action):
#     action_SQ_params, _, SQ_action_subs, _ = SQ_fitting_params(pcd, SQ_type_action)

#     action_part_scale = abs(np.min(SQ_action_subs, axis=0) - np.max(SQ_action_subs, axis=0))
#     action_SQ_params[:3] = action_part_scale
#     return action_SQ_params


def calculate_action_score(reference_params, candidate_params):
    score = 0
    for ref, cand in zip(reference_params, candidate_params):
        shape_diff = sum(abs(cand[3:5] - ref[3:5]))
        position_diff = sum(abs(cand[-3:] - ref[-3:]))
        
        score += shape_diff + position_diff
    return score


def find_best_fit(reference_parts, possible_objects):
    num_parts = len(reference_parts)
    best_score = float('inf')
    best_permutation = None
    best_mapping = []

    # Generate all permutations
    permutations = itertools.permutations(possible_objects, num_parts)

    for perm in permutations:
        candidate_params = np.array(perm)
        score = calculate_action_score(reference_parts, candidate_params)
        if score < best_score:
            best_score = score
            best_permutation = candidate_params
            best_mapping = [(f"Reference Part {i+1}", f"Candidate Object {possible_objects.tolist().index(p.tolist()) + 1}") 
                            for i, p in enumerate(perm)]

    return best_permutation, best_score, best_mapping


# action_SQ_params = calculate_scores_for_pcd(pcd, SQ_type_action)


# Example
reference_parts = np.array([
    [0.1, 0.05, 0.02, 0.0, 1.5, 0.3, 0, 0, 0, 0.1, 0.0, 0.0],  # Head
    [0.02, 0.02, 0.3, 0.0, 1.0, 1.0, 0, 0, 0, 0.0, -0.2, 0.0]   # Stick
])

possible_objects = np.array([
    [0.05, 0.05, 0.02, 0.1, 1.2, 0.6, 0, 0, 0, 0.0, 0.0, 0.0],       # Toroid (Rubber Donut)
    [0.12, 0.06, 0.03, 0.0, 1.6, 0.4, 0.1, 0, 0, 0.11, 0.0, 0.0],   # Rock
    [0.13, 0.06, 0.04, 0.0, 1.8, 0.5, 0, 0, 0, 0.15, 0.0, 0.0],       # Brick
    [0.02, 0.02, 0.28, 0.0, 1.0, 1.0, 0, 0, 0.05, 0.0, -0.21, 0.01], # Metal Rod



])
best_permutation, best_score, best_mapping = find_best_fit(reference_parts, possible_objects)
print("Best Permutation (Matched Candidate Parameters):")
print(best_permutation)
print(f"\nBest Score: {best_score}")
print("\nDetailed Matches:")
for match in best_mapping:
    print(f"{match[0]} matches with {match[1]}")


Best Permutation (Matched Candidate Parameters):
[[ 0.12  0.06  0.03  0.    1.6   0.4   0.1   0.    0.    0.11  0.    0.  ]
 [ 0.02  0.02  0.28  0.    1.    1.    0.    0.    0.05  0.   -0.21  0.01]]

Best Score: 0.13000000000000006

Detailed Matches:
Reference Part 1 matches with Candidate Object 2
Reference Part 2 matches with Candidate Object 4
